In [0]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.4.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

Hit:1 https://repos.azul.com/zulu/deb stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease

debconf: delaying package configuration, since apt-utils is not installed
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-633b4d5d-1eb9-4eed-b363-e352f6281358/bin/python -m pip install --upgrade pip' command.


In [0]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [0]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").config("spark.driver.memory", "2g").getOrCreate()

In [0]:
dbutils.fs.ls("/FileStore/tables/")

Out[4]: [FileInfo(path='dbfs:/FileStore/tables/home_sales_revised/', name='home_sales_revised/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/home_sales_revised.csv', name='home_sales_revised.csv', size=2747874, modificationTime=1728961923000),
 FileInfo(path='dbfs:/FileStore/tables/marathon_df', name='marathon_df', size=1108687, modificationTime=1728269567000),
 FileInfo(path='dbfs:/FileStore/tables/vehicles.csv', name='vehicles.csv', size=115450, modificationTime=1727913288000),
 FileInfo(path='dbfs:/FileStore/tables/vehicles.parquet', name='vehicles.parquet', size=4975246, modificationTime=1727913289000)]

In [0]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
# Load the home_sales_revised.csv file into a DataFrame.
file_location = '/FileStore/tables/home_sales_revised.csv'
df = spark.read.csv(file_location, inferSchema=True, header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [0]:
# 2. Display the DataFrame.
df.createOrReplaceTempView('home_sales')


In [0]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS AVG_Price FROM home_sales  WHERE bedrooms ='4' GROUP BY date_built ORDER BY date_built ASC").show()


+----------+---------+
|date_built|AVG_Price|
+----------+---------+
|      2010|296800.75|
|      2011| 302141.9|
|      2012|298233.42|
|      2013|299999.39|
|      2014|299073.89|
|      2015|307908.86|
|      2016|296050.24|
|      2017|296576.69|
+----------+---------+



In [0]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE bedrooms = '3' AND bathrooms = '3' GROUP BY date_built ORDER BY date_built ASC").show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [0]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE bedrooms = '3' AND bathrooms = '3' AND floors >= '2' AND sqft_living >= '2000' GROUP BY date_built ORDER BY date_built ASC").show()



+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|301130.93|
|      2011|290688.58|
|      2012|319456.36|
|      2013|310510.69|
|      2014| 309821.9|
|      2015| 307147.8|
|      2016|300851.65|
|      2017|293281.74|
+----------+---------+



In [0]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price_per_view_rating
    FROM home_sales 
    GROUP BY view 
    HAVING ROUND(AVG(price), 2) >= 350000 
    ORDER BY view DESC
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------------------+
|view|avg_price_per_view_rating|
+----+-------------------------+
| 100|                1026669.5|
|  99|               1061201.42|
|  98|               1053739.33|
|  97|               1129040.15|
|  96|               1017815.92|
|  95|                1054325.6|
|  94|                1033536.2|
|  93|               1026006.06|
|  92|                970402.55|
|  91|               1137372.73|
|  90|               1062654.16|
|  89|               1107839.15|
|  88|               1031719.35|
|  87|                1072285.2|
|  86|               1070444.25|
|  85|               1056336.74|
|  84|               1117233.13|
|  83|               1033965.93|
|  82|                1063498.0|
|  81|               1053472.79|
+----+-------------------------+
only showing top 20 rows

--- 0.6657025814056396 seconds ---


In [0]:
# 7. Cache the the temporary table home_sales.
cacheDF=df.cache()

In [0]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

Out[45]: True

In [0]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price_per_view_rating
    FROM home_sales 
    GROUP BY view 
    HAVING ROUND(AVG(price), 2) >= 350000 
    ORDER BY view DESC
""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------------------+
|view|avg_price_per_view_rating|
+----+-------------------------+
| 100|                1026669.5|
|  99|               1061201.42|
|  98|               1053739.33|
|  97|               1129040.15|
|  96|               1017815.92|
|  95|                1054325.6|
|  94|                1033536.2|
|  93|               1026006.06|
|  92|                970402.55|
|  91|               1137372.73|
|  90|               1062654.16|
|  89|               1107839.15|
|  88|               1031719.35|
|  87|                1072285.2|
|  86|               1070444.25|
|  85|               1056336.74|
|  84|               1117233.13|
|  83|               1033965.93|
|  82|                1063498.0|
|  81|               1053472.79|
+----+-------------------------+
only showing top 20 rows

--- 0.5834541320800781 seconds ---


In [0]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.parquet("home_sales_partitioned", mode="overwrite", partitionBy="date_built")

In [0]:
dbutils.fs.ls("dbfs:/home_sales_partitioned/")

Out[74]: [FileInfo(path='dbfs:/home_sales_partitioned/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1729020179000),
 FileInfo(path='dbfs:/home_sales_partitioned/_committed_2807342909154149255', name='_committed_2807342909154149255', size=221, modificationTime=1729019243000),
 FileInfo(path='dbfs:/home_sales_partitioned/_committed_4104781885153402075', name='_committed_4104781885153402075', size=123, modificationTime=1729020145000),
 FileInfo(path='dbfs:/home_sales_partitioned/_committed_4995283177445516455', name='_committed_4995283177445516455', size=123, modificationTime=1729020179000),
 FileInfo(path='dbfs:/home_sales_partitioned/_committed_587310410853934407', name='_committed_587310410853934407', size=123, modificationTime=1729019278000),
 FileInfo(path='dbfs:/home_sales_partitioned/_committed_6588580373756024608', name='_committed_6588580373756024608', size=123, modificationTime=1729019225000),
 FileInfo(path='dbfs:/home_sales_partitioned/_committed_8323097846730606236', n

In [0]:
# 11. Read the formatted parquet data.
p_df=spark.read.parquet("dbfs:/home_sales_partitioned/")

In [0]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('home_sales_partitioned')

In [0]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price_per_view_rating
    FROM home_sales_partitioned
    GROUP BY view 
    HAVING ROUND(AVG(price), 2) >= 350000 
    ORDER BY view DESC
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------------------+
|view|avg_price_per_view_rating|
+----+-------------------------+
| 100|                1026669.5|
|  99|               1061201.42|
|  98|               1053739.33|
|  97|               1129040.15|
|  96|               1017815.92|
|  95|                1054325.6|
|  94|                1033536.2|
|  93|               1026006.06|
|  92|                970402.55|
|  91|               1137372.73|
|  90|               1062654.16|
|  89|               1107839.15|
|  88|               1031719.35|
|  87|                1072285.2|
|  86|               1070444.25|
|  85|               1056336.74|
|  84|               1117233.13|
|  83|               1033965.93|
|  82|                1063498.0|
|  81|               1053472.79|
+----+-------------------------+
only showing top 20 rows

--- 2.615440607070923 seconds ---


In [0]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('home_sales')

In [0]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


Out[82]: False